In [1]:
#include "xeus/xcomm.hpp"
#include "xproperty/xobserved.hpp"

#include <string>
#include <iostream>
#include <array>
#include <vector>
#include <type_traits>

inline void xwidget_comm_opened(const xeus::xcomm& comm, const xeus::xmessage& msg)
{
    std::cout << "Hello World" << std::endl;
}

auto& interpreter = xeus::get_interpreter();
interpreter.comm_manager().register_comm_target("jupyter.widget", xwidget_comm_opened);

In [2]:
namespace xeus
{

    template <class D>
    class xobject : public xp::xobserved<D>
    {
    public:

        using base_type = xp::xobserved<D>;
        using derived_type = D;

        inline derived_type& derived_cast() & noexcept
        {
            return *static_cast<derived_type*>(this);
        }

        inline const derived_type& derived_cast() const& noexcept
        {
            return *static_cast<const derived_type*>(this);
        }
        
        inline derived_type derived_cast() && noexcept
        {
            return *static_cast<derived_type*>(this);
        }

        xobject() : m_comm(get_interpreter().comm_manager().target(target_name), xguid())
        {
            set_defaults();
        }

        inline xguid id() const noexcept
        {
            return m_comm.id();
        }

        inline void display() const
        {
            xeus::xjson display_data = R"(
            {
                "application/vnd.jupyter.widget-view+json": {
                    "version_major": "2",
                    "version_minor": "0"
                }
            }
            )"_json;

            display_data["application/vnd.jupyter.widget-view+json"]["model_id"] = 
                xeus::guid_to_hex(this->derived_cast().id());
            
            get_interpreter().display_data(
                std::move(display_data),
                xeus::xjson::object(),
                xeus::xjson::object()
            );
            
            xeus::xjson comm_display = R"(
            {
                "data": {
                    "method": "display"
                }
            }
            )"_json;

            comm_display["comm_id"] =  xeus::guid_to_hex(this->derived_cast().id());

            get_interpreter().comm_manager().target(target_name)->publish_message("comm_msg", xeus::xjson::object(), std::move(comm_display));  
        }

        // TODO: make these property optional (allow none)
        XPROPERTY(std::string, derived_type, _model_module);
        XPROPERTY(std::string, derived_type, _model_module_version);
        XPROPERTY(std::string, derived_type, _model_name);
        XPROPERTY(std::string, derived_type, _view_module);
        XPROPERTY(std::string, derived_type, _view_module_version);
        XPROPERTY(std::string, derived_type, _view_name);

        inline xjson get_state() const
        {
            xjson state;
            state["_model_module"] = _model_module.raw_value();
            state["_model_module_version"] = _model_module_version.raw_value();
            state["_model_name"] = _model_name.raw_value();
            state["_view_module"] = _view_module.raw_value();
            state["_view_module_version"] = _view_module_version.raw_value();
            state["_view_name"] = _view_name.raw_value();
            return state;
        }

    protected:
        
        inline void open()
        {
            m_comm.open(xjson::object(), derived_cast().get_state());
        }

    private:
    
        inline void set_defaults()
        {
            _model_module.raw_value() = "jupyter-js-widgets";
            _model_module_version.raw_value() = "~2.1.4";
            _model_name.raw_value() = "WidgetModel";
            _view_module.raw_value() = "jupyter-js-widgets";
            _view_module_version.raw_value() = "~2.1.4";
            _view_name.raw_value() = "WidgetView";
        }

        xcomm m_comm;

        static const char* target_name;
    };

    // In the cpp (multiple inheritance)
    template <class D>
    const char* xeus::xobject<D>::target_name = "jupyter.widget";
    
    template <class D>
    void to_json(xjson& j, const xobject<D>& o)
    {
        j = "IPY_MODEL_" + guid_to_hex(o.id());
    }
}


In [3]:
namespace xeus
{
    class layout final : public xobject<layout>
    {
    public:

        using base_type = xobject<layout>;

        layout() : base_type()
        {
            set_defaults();
            this->open();
        }

        inline xjson get_state() const
        {
            xjson state = base_type::get_state();
            return state;
        }

    private:

        inline void set_defaults()
        {
            base_type::_model_name.raw_value() = "LayoutModel";
            base_type::_view_name.raw_value() = "LayoutView";
        }
    };
}

In [4]:
namespace xeus
{
    template <class D>
    class xwidget : public xobject<D>
    {
    public:

        using base_type = xobject<D>;
        using derived_type = D;

        xwidget() : base_type()
        {
            set_defaults();
        }
    
        XPROPERTY(::xeus::layout, derived_type, layout);

        inline xjson get_state() const
        {
            xjson state = base_type::get_state();
            state["layout"] = layout.raw_value();
            return state;
        }        
        
    private:

        inline void set_defaults()
        {
            base_type::_model_name.raw_value() = "DOMWidgetModel";
            base_type::_view_name.raw_value() = "DOMWidgetView";
            layout.raw_value() = ::xeus::layout();
        }
    };
}

In [5]:
namespace xeus
{
    template <class T>
    class slider final : public xwidget<slider<T>>
    {
    public:

        using base_type = xwidget<slider<T>>;

        slider() : base_type()
        {
            set_defaults();
            this->open();
        }

        XPROPERTY(double, slider, value);

        inline xjson get_state() const
        {
            xjson state = base_type::get_state();
            state["value"] = value.raw_value();
            return state;
        }

    private:

        inline void set_defaults()
        {
            base_type::_model_name.raw_value() = "FloatSliderModel";
            base_type::_view_name.raw_value() = "FloatSliderView";
        }
    };
}

In [6]:
xeus::slider<double> slider;
slider.display()

In [7]:
slider.display()

In [8]:
slider.display()

In [9]:
namespace xeus
{
    class map final : public xobject<map>
    {
    public:

        using base_type = xobject<map>;
        using location_type = std::array<double, 2>;
        using options_type = std::vector<std::string>;

        map() : base_type()
        {
            set_defaults();
            this->open();
        }

        XPROPERTY(unsigned int, map, max_zoom);
        XPROPERTY(unsigned int, map, min_zoom);
        XPROPERTY(unsigned int, map, zoom);
        XPROPERTY(location_type, map, center);
        XPROPERTY(options_type, map, options);
        
        inline xjson get_state() const
        {
            xjson state = base_type::get_state();
            state["zoom"] = zoom.raw_value();
            state["max_zoom"] = max_zoom.raw_value();
            state["min_zoom"] = min_zoom.raw_value();
            state["center"] = center.raw_value();
            state["options"] = options.raw_value();
            return state;
        }

    private:

        inline void set_defaults()
        {
            _model_module.raw_value() = "jupyter-leaflet";
            _model_module_version.raw_value() = "^0.3.0";        
            _model_name.raw_value() = "LeafletMapModel";
            _view_module.raw_value() = "jupyter-leaflet";
            _view_module_version.raw_value() = "^0.3.0";
            _view_name.raw_value() = "LeafletMapView";
            zoom.raw_value() = 12;
            max_zoom.raw_value() = 20;
            min_zoom.raw_value() = 1;
            center.raw_value() = {0.0, 0.0};
            options.raw_value() = {"attribution_control", "bounce_at_zoom_limits", "box_zoom", "center", "close_popup_on_click", "double_click_zoom", "dragging", "inertia", "inertia_deceleration", "inertia_max_speed", "keyboard", "keyboard_pan_offset", "keyboard_zoom_offset", "max_zoom", "min_zoom", "scroll_wheel_zoom", "tap", "tap_tolerance", "touch_zoom", "world_copy_jump", "zoom", "zoom_animation_threshold", "zoom_control", "zoom_start"};
        }
    };
}

In [10]:
xeus::map m;
m.display()